In [ ]:
!pip freeze | grep scikit-learn

In [1]:
from pathlib import Path
import pickle
import pandas as pd

In [9]:
year = 2021
month = 2
models_path = Path().cwd() / "models"
output_file = Path().cwd() / "results" / "df_result.parquet"

In [10]:
with open(models_path / 'model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [11]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [13]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [14]:
y_pred.mean()

16.191691679979066

In [15]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
df_result = pd.DataFrame({"ride_id": df["ride_id"], "predictions": y_pred})

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False,
)